In [5]:
import pandas as pd
import numpy as np
# import joblib
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# from imblearn.under_sampling import RandomUnderSampler

In [6]:
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("kaiyoo88/fake-real-estate")

path = "/kaggle/input/opeb-s"
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/opeb-s


In [9]:
# f-string: f"{path}/train.csv"

# 데이터 로드
train = pd.read_csv(f'{path}/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/opeb-s/train.csv'

In [ ]:
# Feature & Target 설정
x = train.drop(['ID', '허위매물여부'], axis=1)
y = train['허위매물여부']

## 1) 결측치 처리: KNN Imputer 사용 (더 정밀한 방식)

In [ ]:
# 1) 결측치 처리: KNN Imputer 사용 (더 정밀한 방식)
knn_imputer = KNNImputer(n_neighbors=5)  # K=5로 설정하여 결측치 예측
columns_fill_knn = ['해당층', '총층', '전용면적', '방수', '욕실수', '총주차대수']
x[columns_fill_knn] = knn_imputer.fit_transform(x[columns_fill_knn])

## 2) Feature Engineering 추가

In [ ]:
## 2) Feature Engineering 추가

# 1 단위면적당 가격 (㎡당 가격)
x['단위면적당가격'] = x['보증금'] / x['전용면적']
x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)

# 2 보증금 대비 월세 비율
x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
x['보증금_월세비율'].fillna(x['보증금_월세비율'].median(), inplace=True)

# 3 층수 비율 (해당층 / 총층)
x['층수_비율'] = x['해당층'] / x['총층']
x['층수_비율'].fillna(x['층수_비율'].median(), inplace=True)

# 4 게재일 관련 Feature
x['게재일'] = pd.to_datetime(x['게재일'])
x['게재요일'] = x['게재일'].dt.weekday
x['게재일_경과일'] = (datetime(2025, 1, 20) - x['게재일']).dt.days

# 5 방향 그룹화
direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
x['방향_그룹'] = x['방향'].map(direction_map)

# 6 이상 가격 탐지 Feature
unit_price_mean = x['단위면적당가격'].mean()
unit_price_std = x['단위면적당가격'].std()
x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()

# 7 주차 가능 여부 수치 변환
x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})

# 8 월세 + 관리비 총 비용
x['월세_총비용'] = x['월세'] + x['관리비']
x['월세_총비용'].fillna(x['월세_총비용'].median(), inplace=True)

# 9 관리비 비율 (관리비 / 월세)
x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
x['관리비_비율'].fillna(x['관리비_비율'].median(), inplace=True)

# 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)

# 11 플랫폼별 평균 보증금 / 월세 차이
플랫폼_보증금평균 = train.groupby('제공플랫폼')['보증금'].mean()
플랫폼_월세평균 = train.groupby('제공플랫폼')['월세'].mean()

x['제공플랫폼_보증금차이'] = x['보증금'] - x['제공플랫폼'].map(플랫폼_보증금평균)
x['제공플랫폼_월세차이'] = x['월세'] - x['제공플랫폼'].map(플랫폼_월세평균)


## 3) Label Encoding (문자열 데이터를 숫자로 변환)

In [ ]:
## 3) Label Encoding (문자열 데이터를 숫자로 변환)
label_encode_cols = ['중개사무소', '게재일', '제공플랫폼', '방향', '방향_그룹']
label_encoders = {}
for col in label_encode_cols:
    le = LabelEncoder()
    x[col] = le.fit_transform(x[col].astype(str))
    label_encoders[col] = le  # 나중에 변환을 위해 저장

## 4) One-Hot Encoding 적용

In [ ]:
# 4) One-Hot Encoding 적용
one_hot_cols = ['매물확인방식', '주차가능여부']
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
x_encoded = one_hot_encoder.fit_transform(x[one_hot_cols])
x_encoded_df = pd.DataFrame(x_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=x.index)

# 기존 데이터와 병합 후 기존 열 삭제
x = pd.concat([x.drop(columns=one_hot_cols), x_encoded_df], axis=1)

## 5) Train / Validation 분할 (Stratified 방식)

In [8]:
# 5) Train / Validation 분할 (Stratified 방식)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

NameError: name 'x' is not defined

## 6) 노이즈 적용 + 소수 데이터 증강

In [ ]:
def add_noise(df, noise_level=0.05):
    numeric_cols = df.select_dtypes(include=[np.number]).columns  # 숫자형 컬럼만 선택
    df[numeric_cols] = df[numeric_cols] * (1 + noise_level * np.random.randn(*df[numeric_cols].shape))
    return df

print("Before Noise Injection:", len(x_train))
x_train = add_noise(x_train)
print("After Noise Injection:", len(x_train))


In [ ]:
def augment_minority_class(x_train, y_train, num_augments=2, noise_level=0.05):
    numeric_cols = x_train.select_dtypes(include=[np.number]).columns
    x_train_1 = x_train[y_train == 1].copy()  # 소수 클래스(1)만 선택
    augmented_data = []

    for _ in range(num_augments):
        x_aug = x_train_1.copy()
        x_aug[numeric_cols] = x_aug[numeric_cols] * (1 + noise_level * np.random.randn(*x_aug[numeric_cols].shape))
        augmented_data.append(x_aug)

    x_train = pd.concat([x_train] + augmented_data, axis=0).reset_index(drop=True)
    y_train = np.concatenate([y_train] + [np.ones(len(x_train_1))] * num_augments)  # 레이블 추가

    return x_train, y_train

print("Before Minority Augmentation:", len(x_train))
x_train, y_train = augment_minority_class(x_train, y_train, num_augments=2)
print("After Minority Augmentation:", len(x_train))


## 7) XGBoost 모델 정의 + 불균형 보정 옵션 적용

In [ ]:
# XGBoost 모델 정의 + 불균형 보정 옵션 적용
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])  # 0의 개수 / 1의 개수 

# XGBClassifier 객체를 명확하게 정의
xgb_model = XGBClassifier(scale_pos_weight=scale_pos_weight)

## 8) 하이퍼파라미터 튜닝

In [ ]:
# GridSearchCV 설정
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [6, 10],
    'learning_rate': [0.01, 0.1],
    'gamma': [0, 0.1, 0.5]
}

grid_search = GridSearchCV(
    estimator=xgb_model,  # ✅ 명확하게 정의한 모델을 전달
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train, y_train)

# 6. 최적의 하이퍼파라미터 출력
print("Best parameters:", grid_search.best_params_)
print("Best Macro F1-score:", grid_search.best_score_)

## 9) 최적 모델 학습 및 평가

In [ ]:
# 최적 모델 학습 및 평가
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(x_val)

macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Validation Macro F1-score: {macro_f1:.4f}")

# 예측하기

In [ ]:
# Test 데이터 로드
test = pd.read_csv(f'{path}/test.csv')

In [ ]:
# Test 결측값 대체
test[columns_fill_knn] = knn_imputer.transform(test[columns_fill_knn])

In [ ]:
def create_new_features(x): 
    # 1 단위면적당 가격 (㎡당 가격)
    x['단위면적당가격'] = x['보증금'] / x['전용면적']
    x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)
    
    # 2 보증금 대비 월세 비율
    x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
    x['보증금_월세비율'].fillna(x['보증금_월세비율'].median(), inplace=True)
    
    # 3 층수 비율 (해당층 / 총층)
    x['층수_비율'] = x['해당층'] / x['총층']
    x['층수_비율'].fillna(x['층수_비율'].median(), inplace=True)
    
    # 4 게재일 관련 Feature
    x['게재일'] = pd.to_datetime(x['게재일'])
    x['게재요일'] = x['게재일'].dt.weekday
    x['게재일_경과일'] = (datetime(2025, 1, 20) - x['게재일']).dt.days
    
    # 5 방향 그룹화
    direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
    x['방향_그룹'] = x['방향'].map(direction_map)
    
    # 6 이상 가격 탐지 Feature
    unit_price_mean = x['단위면적당가격'].mean()
    unit_price_std = x['단위면적당가격'].std()
    x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()
    
    # 7 주차 가능 여부 수치 변환
    x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})
    
    # 8 월세 + 관리비 총 비용
    x['월세_총비용'] = x['월세'] + x['관리비']
    x['월세_총비용'].fillna(x['월세_총비용'].median(), inplace=True)
    
    # 9 관리비 비율 (관리비 / 월세)
    x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
    x['관리비_비율'].fillna(x['관리비_비율'].median(), inplace=True)
    
    # 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
    x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
    x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)
    
    # 11 플랫폼별 평균 보증금 / 월세 차이
    플랫폼_보증금평균 = train.groupby('제공플랫폼')['보증금'].mean()
    플랫폼_월세평균 = train.groupby('제공플랫폼')['월세'].mean()
    
    x['제공플랫폼_보증금차이'] = x['보증금'] - x['제공플랫폼'].map(플랫폼_보증금평균)
    x['제공플랫폼_월세차이'] = x['월세'] - x['제공플랫폼'].map(플랫폼_월세평균)
    return x

test = create_new_features(test)
test.head()

In [ ]:
# Label Encoding 
for col in label_encode_cols:
    if col in test.columns:
        le = label_encoders[col] 
        test[col] = test[col].astype(str)
        unseen = set(test[col].unique()) - set(le.classes_) 
        # unseen = []

        if unseen: # 뜬금포가 있다
            le.classes_ = np.append(le.classes_, list(unseen))
        test[col] = le.transform(test[col].astype(str))

In [ ]:
# One-Hot Encoding
test_encoded = one_hot_encoder.transform(test[one_hot_cols])
test_encoded_df = pd.DataFrame(test_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=test.index)

test = pd.concat([test.drop(columns=one_hot_cols), test_encoded_df], axis=1)

In [ ]:
test.drop(columns=['ID'],inplace=True)

In [ ]:
pred = pd.Series(best_model.predict(test))

# 제출하기

In [ ]:
submit = pd.read_csv(f'{path}/sample_submission.csv')

In [ ]:
submit['허위매물여부'] = pred # 우리의 예측 넣는다
submit.head()

In [ ]:
submit.to_csv('./baseline_submission.csv',index=False)